# **Розширений план-контент групового заняття**  
## **Тема 3. Заняття 4. Особливості використання інформаційних технологій для аналізу часових рядів**  
### **1. Особливості підготовки даних для моделювання часових рядів**  
### **2. Особливості моделювання часових рядів**  

---

## **1. Вступ (10 хв)**  

### **1.1. Мета заняття**  
- Ознайомити слухачів з особливостями підготовки даних для аналізу часових рядів.  
- Навчити базовим методам аналізу часових рядів у військових аналітичних системах.  
- Розглянути сучасні інструменти та алгоритми моделювання часових рядів.  
- Виконати практичні завдання з аналізу часових рядів у Python.  

### **1.2. Очікувані результати**  
Після заняття слухачі зможуть:  
✅ Розуміти основні поняття часових рядів та їх структуру.  
✅ Виконувати очищення та підготовку даних для аналізу.  
✅ Використовувати алгоритми прогнозування часових рядів (ARIMA, LSTM).  
✅ Оцінювати ефективність моделей часових рядів.  

---

## **2. Особливості підготовки даних для моделювання часових рядів (40 хв)**  

### **2.1. Що таке часовий ряд?**  
📌 **Часовий ряд** – це послідовність даних, що змінюється у часі.  
🔹 **Приклади часових рядів у військовій аналітиці:**  
- Частота бойових зіткнень у регіонах.  
- Кількість атак ворога за день/місяць.  
- Витрати ресурсів на військові операції.  

📌 **Компоненти часових рядів:**  
1️⃣ **Тренд** – довгострокова зміна рівня ряду.  
2️⃣ **Сезонність** – періодичні коливання.  
3️⃣ **Циклічність** – зміни, що залежать від зовнішніх факторів.  
4️⃣ **Шум** – випадкові зміни.  

---

### **2.2. Очищення та підготовка даних**  
📌 **Кроки підготовки даних:**  
1️⃣ **Заповнення пропущених значень**  
2️⃣ **Агрегація та нормалізація**  
3️⃣ **Визначення тренду та сезонності**  
4️⃣ **Перетворення у формат для машинного навчання**  

📌 **Приклад очищення даних у Python**  
```python
import pandas as pd

# Завантаження даних
df = pd.read_csv("combat_reports.csv", parse_dates=["date"])

# Перевірка на пропущені значення
df.fillna(method="ffill", inplace=True)

# Агрегація за місяцями
df["month"] = df["date"].dt.to_period("M")
df_grouped = df.groupby("month")["num_casualties"].sum().reset_index()

print(df_grouped.head())
```

📌 **Очікуваний результат:**  
```
   month  num_casualties
0 2024-01       150
1 2024-02       175
2 2024-03       130
```

---

## **3. Особливості моделювання часових рядів (50 хв)**  

### **3.1. Основні методи аналізу часових рядів**  
📌 **Статистичні методи:**  
- **Ковзне середнє (Moving Average)** – згладжування ряду.  
- **Автокореляція (ACF, PACF)** – пошук залежностей.  
- **ARIMA (AutoRegressive Integrated Moving Average)** – прогнозування на основі минулих даних.  

📌 **Методи машинного навчання:**  
- **Random Forest Regressor** – прогнозування на основі історичних даних.  
- **LSTM (Long Short-Term Memory)** – нейромережі для прогнозування часових рядів.  

---

### **3.2. Аналіз та прогнозування за допомогою ARIMA**  
📌 **Приклад прогнозування кількості атак ARIMA**  
```python
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Завантаження даних
df = pd.read_csv("combat_reports.csv", parse_dates=["date"])
df.set_index("date", inplace=True)
df = df.resample("M").sum()

# Створення моделі ARIMA
model = ARIMA(df["num_casualties"], order=(2, 1, 2))
model_fit = model.fit()

# Прогноз на 6 місяців вперед
forecast = model_fit.forecast(steps=6)
print(forecast)

# Побудова графіка
plt.figure(figsize=(10,5))
plt.plot(df.index, df["num_casualties"], label="Фактичні дані")
plt.plot(pd.date_range(df.index[-1], periods=7, freq="M")[1:], forecast, label="Прогноз", linestyle="dashed")
plt.legend()
plt.show()
```

📌 **Очікуваний результат:**  
- Побудова прогнозу на 6 місяців вперед.  
- Графік зіставлення реальних і прогнозованих значень.  

---

### **3.3. Прогнозування часових рядів за допомогою LSTM**  
📌 **LSTM – нейромережі для обробки часових рядів**  

```python
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Підготовка даних
df["num_casualties"] = df["num_casualties"].astype(float)
data = df["num_casualties"].values.reshape(-1, 1)

# Нормалізація даних
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Формування набору для навчання
X_train, y_train = [], []
for i in range(10, len(data_scaled)):
    X_train.append(data_scaled[i-10:i])
    y_train.append(data_scaled[i])

X_train, y_train = np.array(X_train), np.array(y_train)

# Побудова LSTM-моделі
model = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(10, 1)),
    LSTM(50, activation="relu"),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.fit(X_train, y_train, epochs=20, batch_size=16)

# Прогноз
y_pred = model.predict(X_train[-1].reshape(1, 10, 1))
y_pred = scaler.inverse_transform(y_pred)
print(f"Прогнозований рівень атак: {y_pred[0][0]}")
```

📌 **Очікуваний результат:**  
- Модель LSTM навчається прогнозувати майбутні значення атак.  
- Вивід прогнозованого рівня атак.  

---

## **4. Практичні завдання (30 хв)**  
✅ **Завдання 1:** Очистити часовий ряд від пропущених значень та візуалізувати тренд.  
✅ **Завдання 2:** Виконати прогноз на 6 місяців вперед за допомогою ARIMA.  
✅ **Завдання 3:** Використати LSTM для прогнозування наступного місяця.  

---

## **5. Висновки та підсумки заняття (10 хв)**  
✅ **Основні висновки:**  
- Часові ряди складаються з тренду, сезонності та шуму.  
- Підготовка даних включає очищення, нормалізацію та агрегацію.  
- ARIMA добре працює з короткими рядами, LSTM – з великими.  

🚀 **Ключовий меседж:**  
**Аналіз часових рядів допомагає прогнозувати події у військовій аналітиці, що підвищує ефективність ухвалення рішень!**

[Приклад](https://www.kaggle.com/code/vbmokin/crypto-btc-advanced-analysis-forecasting)